In [19]:
import boto3
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import datetime

In [2]:
with open("AWS_credentials.txt", "r") as creds:
    creds_list = json.loads(creds.read())

In [3]:
s3 = boto3.resource('s3', aws_access_key_id=creds_list[0], aws_secret_access_key=creds_list[1])
my_bucket = s3.Bucket('hadis-s3-project-bucket')

In [4]:
for object_summary in my_bucket.objects.filter(Prefix="Real-Estate/input/"):
    print(object_summary.key)

Real-Estate/input/
Real-Estate/input/2024-04-19_6901.parquet.gzip
Real-Estate/input/2024-04-30_6901.parquet.gzip


In [5]:
my_bucket.download_file("Real-Estate/input/2024-04-30_6901.parquet.gzip", "2024-04-30_6901.parquet.gzip")

In [6]:
my_spark = SparkSession.builder.getOrCreate()

In [7]:
print(my_spark.catalog.listTables())

[]


In [68]:
path = '2024-04-30_6901.parquet.gzip'
df = my_spark.read.format('parquet').options(header=True,inferSchema=True).load(path)

In [69]:
df.show()

+-------+-------+--------------+--------+------+--------------------+--------------------+--------+---------+--------+------------+------------+-------------+---------------+------------+-------------+--------------------+--------------------+-------------------------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+-------------+-----+-----+--------------+-----+--------+--------+--------+--------------------+----------+----------+-------------+--------------------+--------------------+--------------------+-------+--------------------+----------+--------------------+-------------+--------------+-----------------+-----------------+-----------------+-----------+----------------+--------+--------------------+--------------------+--------------------+---------------------------------+--------------------+------------------+--------------+-----

In [70]:
unreq_cols = ["productLabel","rentFrequency","projectNumber","title_l1","title_l2","title_l3","coverPhoto","photoCount","videoCount","panoramaCount","randBoostScore","randBoostScore_l1",
              "randBoostScore_l2","randBoostScore_l3","floorPlanID","amenities_l1","amenities_l2","amenities_l3","indyScore","indyScore_l1","indyScore_l2","indyScore_l3","photoIDs",
              "coverVideo"]
x = datetime.datetime.timestamp(datetime.datetime.now() - datetime.timedelta(days=1))
df = df.drop(*unreq_cols).filter((df.createdAt > x) | (df.updatedAt > x))

In [71]:
df.show()

+-------+-------+--------------+--------+------+--------------------+--------------------+--------+---------+--------+------------+---------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+-------------+-----+-----+--------------+-----+--------+--------+--------+--------------------+--------------------+--------------------+-------+--------------------+----------+--------------------+-------------+----------------+--------+--------------------+--------------------+------------------+--------------+---------------------+---------+-------------------+--------+--------+--------------------+--------------+--------------------+--------------------+--------------------+---------------------+----------------+------------------+---------------+
|     id|ownerID|userExternalID|sourceID| state|             _geoloc|           geography| purpo

In [72]:
df = df.withColumn("createdAt_float", df["createdAt"].cast("float"))

In [78]:
df.show()

+-------+-------+--------------+--------+------+--------------------+--------------------+--------+---------+--------+------------+---------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+-------------+-----+-----+--------------+-----+--------+--------+--------+--------------------+--------------------+--------------------+-------+--------------------+----------+--------------------+-------------+----------------+--------+--------------------+--------------------+------------------+--------------+---------------------+---------+-------------------+--------+--------+--------------------+--------------+--------------------+--------------------+--------------------+---------------------+----------------+------------------+---------------+---------------+
|     id|ownerID|userExternalID|sourceID| state|             _geoloc|           

In [80]:
def time_fix(time_col):
    return datetime.datetime.fromtimestamp(time_col).strftime('%Y-%m-%d %H:%M:%S')
udf_func = udf(lambda time: time_fix(time), StringType())
df = df.withColumn("created_at", udf_func(df.createdAt_float))

In [81]:
df.show()

Py4JJavaError: An error occurred while calling o354.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 35.0 failed 1 times, most recent failure: Lost task 0.0 in stage 35.0 (TID 35) (DESKTOP-VS50GFG executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(

In [ ]:
import time 
print(time.time())
print(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

In [77]:
type(df.createdAt_float)

pyspark.sql.column.Column

In [65]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- ownerID: string (nullable = true)
 |-- userExternalID: string (nullable = true)
 |-- sourceID: long (nullable = true)
 |-- state: string (nullable = true)
 |-- _geoloc: struct (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lng: double (nullable = true)
 |-- geography: struct (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lng: double (nullable = true)
 |-- purpose: string (nullable = true)
 |-- price: double (nullable = true)
 |-- product: string (nullable = true)
 |-- productScore: long (nullable = true)
 |-- referenceNumber: string (nullable = true)
 |-- permitNumber: string (nullable = true)
 |-- title: string (nullable = true)
 |-- externalID: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- slug_l1: string (nullable = true)
 |-- slug_l2: string (nullable = true)
 |-- slug_l3: string (nullable = true)
 |-- location: array (nullable = true)
 |    |-- element: struct (containsNull = tr

In [84]:
type(datetime.datetime.utcfromtimestamp(1.71445837E9).strftime('%Y-%m-%d %H:%M:%S'))

C:\Users\hadi hashemi\AppData\Local\Temp\ipykernel_15392\1152456848.py:1: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  type(datetime.datetime.utcfromtimestamp(1.71445837E9).strftime('%Y-%m-%d %H:%M:%S'))


str

In [79]:
time_fix(df.createdAt_float)

TypeError: 'Column' object cannot be interpreted as an integer